# Warp a disk

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
%matplotlib widget

In [ ]:
from importlib import reload
import helper
reload(helper)

## General Definitions of disk surface and warp parameters

Define surface:

$z = h_1 \, r^{1+hr\_index}$

In [ ]:
h1 = 0.05
hr_index = 0.25

Calculate the interfaces and cell centers of the disk surface

In [ ]:
surf = helper.get_surface(h1=h1, hr_index=hr_index)
points = surf['points']
points_c = surf['points_c']
r = surf['r']
phic = surf['phic']
nr, nphi = points.shape[:-1]

Define the warp (inclination for each ring): logistic function from $i=0$ outside transitioning to `warp_max` at radius `r0` over a transition width of `dr`.

In [ ]:
warp_max = np.pi / 6
dr = 0.1
r0 = 1

In [ ]:
# calculate warp inclinations
warp   = warp_max / (1.0 + np.exp((r[:, 0] - r0) / dr))
warp_c = 0.5 * (warp[1:] + warp[:-1])

Calculate the velocities of all centers

In [ ]:
v = (points_c[:, :, 0]**2 + points_c[:, :, 1]**2)**-0.25
v = v[None, :, :] * np.array([-np.sin(phic), np.cos(phic), np.zeros_like(phic)])
v = np.moveaxis(v, 0, 2)

Define each radius' twist

In [ ]:
twist_0 = -np.pi * 3 / 4 # this is the general rotation of the entire disk around the original axis
twist = np.linspace(0, 20, nr) * np.pi / 180. + twist_0
twist_c = 0.5 * (twist[1:] + twist[:-1]) # twist of the cell centers

define the inclination of the observer as rotation around x

In [ ]:
inc_obs = 65 / 180 * np.pi
PA_obs = 0 / 180 * np.pi

In [ ]:
points2 = helper.warp(points, warp, twist, inc=inc_obs, PA=PA_obs)
points2_c = helper.warp(points_c, warp_c, twist_c, inc=inc_obs, PA=PA_obs)
v2 = helper.warp(v, warp_c, twist_c, inc=inc_obs, PA=PA_obs)

In [ ]:
f, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.axis('off')

_x, _y, _ = points2.T
ax.pcolormesh(_x, _y, Normalize()(v2[:, :, -1]).T, cmap=plt.cm.RdBu_r, ec='k')

_x, _y, _ = points2_c.T
ax.quiver(_x, _y, v2[:, :, 0].T, v2[:, :, 1].T)

In [ ]:
f.savefig('warp.pdf', transparent=True, bbox_inches='tight')